In [5]:
import pandas as pd
from datetime import datetime, timedelta
import os

In [6]:
input_path = "/root/lanyun-tmp/prediction/UserBehavior.csv"
output_path = "/root/lanyun-tmp/prediction/train.csv"
last_day = datetime(2017, 12, 1, 23, 59, 59) 
required_days = 4
batch_size = 50000
tmp_folder = "./tmp_batches"

In [17]:
print("🚀 加载数据...")
df = pd.read_csv(input_path, header=None,
                 names=["user_id", "item_id", "category_id", "behavior_type", "timestamp"])
df["time"] = pd.to_datetime(df["timestamp"], unit="s")
df = df[df["time"] <= last_day].copy()
df = df.sort_values("time")

🚀 加载数据...


Train & Validation:

In [ ]:
# === 筛选在 last_day 前发生的所有行为记录 ===
used_df = df[df["time"] <= last_day].copy()

# === 获取每个用户最后一次活跃时间 ===
print("🔍 判断哪些用户在最后一天前就已连续 ≥ N 天无行为...")
last_active = used_df.groupby("user_id")["time"].max()
event_cutoff_time = last_day - timedelta(days=required_days)

# 如果最后活跃时间 ≤ 截止时间 ⇒ 表明从 cutoff 后用户就再也没有行为 ⇒ event=1
event_series = (last_active <= event_cutoff_time).astype(int).rename("event")

# === 特征统计：duration 和行为类型 ===
print("📊 计算用户特征（duration + 行为分布）...")
user_span = used_df.groupby("user_id")["time"].agg(["min", "max"]).reset_index()
user_span["duration"] = (user_span["max"] - user_span["min"]).dt.days
user_span = user_span[["user_id", "duration"]]

features = used_df.groupby("user_id")["behavior_type"].value_counts().unstack().fillna(0).reset_index()

# === 合并保存 ===
print("💾 合并并保存最终训练数据...")
cox_df = user_span.merge(event_series, on="user_id").merge(features, on="user_id", how="left")
cox_df.to_csv(output_path, index=False)

# === 输出统计 ===
print(f"✅ 已保存：{output_path}")
print(f"✅ 流失用户数 (event=1): {(cox_df['event'] == 1).sum()} / {len(cox_df)}")

Test:

In [ ]:

# === 创建临时文件夹存批次结果 ===
os.makedirs(tmp_folder, exist_ok=True)

# === 获取所有用户并分批处理 ===
user_ids = df["user_id"].unique()
print(f"📦 共 {len(user_ids)} 个用户，将分批处理每批 {batch_size} 个用户")

for i in range(0, len(user_ids), batch_size):
    batch_user_ids = user_ids[i:i+batch_size]
    print(f"\n🚧 正在处理批次 {i//batch_size + 1}（用户 {i} ~ {i+len(batch_user_ids)-1}）...")

    # 当前批次用户的数据
    batch_df = df[df["user_id"].isin(batch_user_ids)].copy()

    # 获取最后活跃时间
    last_active = batch_df.groupby("user_id")["time"].max()
    event_cutoff_time = last_day - timedelta(days=required_days)
    event_series = (last_active <= event_cutoff_time).astype(int).rename("event")

    # duration 统计
    user_span = batch_df.groupby("user_id")["time"].agg(["min", "max"]).reset_index()
    user_span["duration"] = (user_span["max"] - user_span["min"]).dt.days
    user_span = user_span[["user_id", "duration"]]

    # 行为类型统计
    features = batch_df.groupby("user_id")["behavior_type"].value_counts().unstack().fillna(0).reset_index()

    # 合并
    cox_df = user_span.merge(event_series, on="user_id").merge(features, on="user_id", how="left")

    # 保存临时结果
    tmp_file = os.path.join(tmp_folder, f"tmp_batch_{i//batch_size + 1}.csv")
    cox_df.to_csv(tmp_file, index=False)
    print(f"✅ 批次 {i//batch_size + 1} 已保存：{tmp_file}")

# === 合并所有批次结果 ===
print("\n🔗 合并所有批次文件...")
all_parts = [pd.read_csv(os.path.join(tmp_folder, f)) for f in sorted(os.listdir(tmp_folder)) if f.endswith(".csv")]
final_df = pd.concat(all_parts, ignore_index=True)
final_df.to_csv(output_path, index=False)

print(f"\n✅ 最终训练集已保存：{output_path}")
print(f"✅ 流失用户数 (event=1): {(final_df['event'] == 1).sum()} / {len(final_df)}")

In [18]:
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler

In [19]:
# === 特征构建函数 ===
def add_user_features(df: pd.DataFrame, window_end_date: str, output_path: str, df_all_behavior: pd.DataFrame):
    df = df.copy()
    df["total_actions"] = df[["pv", "cart", "fav", "buy"]].sum(axis=1)
    df["action_days"] = df["duration"] + 1
    df["avg_actions_per_day"] = df["total_actions"] / (df["action_days"] + 1e-5)

    def compute_entropy(row):
        actions = row[["pv", "cart", "fav", "buy"]].values
        probs = actions / (actions.sum() + 1e-5)
        return entropy(probs)

    df["behavior_entropy"] = df.apply(compute_entropy, axis=1)

    # 计算 recency（仅限截止日期前的行为）
    cutoff = pd.to_datetime(window_end_date)
    last_time = df_all_behavior[df_all_behavior["time"] <= cutoff].groupby("user_id")["time"].max().reset_index()
    last_time["recency"] = (cutoff - last_time["time"]).dt.days
    df = df.merge(last_time[["user_id", "recency"]], on="user_id", how="left")
    df["recency"] = df["recency"].fillna(df["duration"])

    # 特征归一化
    cols_to_scale = ["duration", "total_actions", "action_days", "avg_actions_per_day", "behavior_entropy", "recency"]
    scaler = MinMaxScaler()
    df[[col + "_scaled" for col in cols_to_scale]] = scaler.fit_transform(df[cols_to_scale])

    # 保存文件
    df.to_csv(output_path, index=False)
    print(f"✅ 特征已保存至：{output_path}，包含用户数：{len(df)}")
    return df



In [20]:
# === 执行构建：Train / Val / Test ===
train_df = pd.read_csv("data/train_u.csv")
val_df = pd.read_csv("data/valid_u.csv")
test_df = pd.read_csv("data/test_u.csv")

train_out = add_user_features(train_df, "2017-12-01", "cox_train_with_features.csv", df)
val_out = add_user_features(val_df, "2017-12-02", "cox_val_with_features.csv", df)
test_out = add_user_features(test_df, "2017-12-03", "cox_test_with_features.csv", df)

# 示例输出
print(train_out.head())

✅ 特征已保存至：cox_train_with_features.csv，包含用户数：987984
✅ 特征已保存至：cox_val_with_features.csv，包含用户数：987992
✅ 特征已保存至：cox_test_with_features.csv，包含用户数：987994
   user_id  duration  event  buy  cart  fav     pv  total_actions  \
0        1         7      0  0.0   0.0  0.0   44.0           44.0   
1        2         5      0  3.0   0.0  7.0   27.0           37.0   
2        3         6      0  0.0  14.0  5.0   22.0           41.0   
3        4         6      0  4.0  13.0  0.0  248.0          265.0   
4        5         4      0  0.0   0.0  0.0   58.0           58.0   

   action_days  avg_actions_per_day  behavior_entropy  recency  \
0            8             5.499993          0.000000      0.0   
1            6             6.166656          0.748626      1.0   
2            7             5.857134          0.957549      0.0   
3            7            37.857089          0.273240      0.0   
4            5            11.599977          0.000000      0.0   

   duration_scaled  total_actions_scaled 

In [21]:
# === 输出每个数据集中 event=1 的用户数量和比例 ===
for name, df in zip(["Train", "Val", "Test"], [train_out, val_out, test_out]):
    event1 = (df["event"] == 1).sum()
    total = len(df)
    print(f"{name} 集合: event=1 数量 = {event1} / {total} ({event1/total:.2%})")


Train 集合: event=1 数量 = 20716 / 987984 (2.10%)
Val 集合: event=1 数量 = 4495 / 987992 (0.45%)
Test 集合: event=1 数量 = 2169 / 987994 (0.22%)
